In [1]:
%load_ext autoreload
%autoreload 2


In [2]:
import os
os.chdir('/home/maksym/advbench')
# TODO: a temporary hack. Later on rather use `pip install`.



First install **`AdvBench`**:



In [ ]:
!pip install -q git+https://github.com/fra31/advbench
!pip install -q -r torch==1.4.0 requests 



Now let's see how many models are available in the **Model Zoo**:



In [3]:
from model_zoo.models import model_dicts
models = model_dicts.keys()
print('In total {} models: {}'.format(len(models), models))



In total 9 models: odict_keys(['Carmon2019Unlabeled', 'Sehwag2020Hydra', 'Wang2020Improving', 'Hendrycks2019Using', 'Rice2020Overfitting', 'Zhang2019Theoretically', 'Engstrom2019Robustness', 'Chen2020Adversarial', 'Huang2020Self'])


Now let's try to restore the most robust model from [Carmon2019Unlabeled](https://arxiv.org/abs/1905.13736) that 
achieves 59.50% evaluated with AA+:



In [4]:
from data import load_cifar10
from utils import load_model, clean_accuracy

x_test, y_test = load_cifar10(n_examples=100)
model = load_model(model_name='Carmon2019Unlabeled').cuda().eval()

acc = clean_accuracy(model=model, x=x_test, y=y_test, batch_size=128)
print('Clean accuracy: {:.2%}'.format(acc))



Files already downloaded and verified
Clean accuracy: 89.00%


89% clean accuracy is quite reasonable which means that we have restored the model successfully! \
Now let's try to evaluate its robustness with a cheap version [AutoAttack](https://arxiv.org/abs/2003.01690) from 
ICML'20 (we use the full version of AutoAttack for our standardized evaluation of Linf-robustness):



In [ ]:
from attacks.autoattack import AutoAttack
adversary = AutoAttack(model, norm='Linf', eps=8/255, plus=False)
adversary.cheap()
x_adv = adversary.run_standard_evaluation(x_test, y_test)



You can also easily plug in any existing library with adversarial attacks such as [FoolBox](https://github.com/bethgelab/foolbox):



In [ ]:
!pip install -q foolbox 



In [ ]:
import foolbox as fb
fmodel = fb.PyTorchModel(model, bounds=(0, 1))

_, advs, success = fb.attacks.LinfPGD()(fmodel, images, labels, epsilons=[8/255])
